In [ ]:
import os
import json
import pandas as pd
import numpy as np
import shutil
from tqdm import tqdm

def regenerate_metadata_from_csv(csv_path, base_folder):
    """
    Regenera los archivos metadata.jsonl para cada conjunto basándose en las imágenes existentes
    y los datos del CSV original.
    
    Args:
        csv_path: Ruta al archivo CSV con los datos originales
        base_folder: Carpeta base del proyecto
    """
    # Cargar el CSV original
    df = pd.read_csv(csv_path)
    
    # Crear un índice por nombre de archivo para búsqueda rápida
    df_indexed = df.set_index("Archivo_Imagen")
    
    # Definir las rutas de las carpetas
    sets_folder = os.path.join(base_folder, "sets")
    set_folders = ["train", "val", "test"]
    
    print("Regenerando metadatos para todos los conjuntos...")
    
    for set_name in set_folders:
        set_folder = os.path.join(sets_folder, set_name)
        metadata_path = os.path.join(set_folder, "metadata.jsonl")
        
        # Verificar que la carpeta existe
        if not os.path.exists(set_folder):
            print(f"La carpeta {set_folder} no existe. Saltando...")
            continue
            
        # Eliminar el archivo metadata.jsonl existente
        if os.path.exists(metadata_path):
            os.remove(metadata_path)
            
        # Obtener la lista de archivos de imagen en la carpeta
        image_files = [f for f in os.listdir(set_folder) 
                      if os.path.isfile(os.path.join(set_folder, f)) 
                      and f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        print(f"Procesando {len(image_files)} imágenes en {set_name}...")
        
        # Crear el nuevo archivo metadata.jsonl
        for file_name in tqdm(image_files, desc=f"Regenerando metadatos para {set_name}"):
            try:
                # Buscar datos en el CSV original
                if file_name in df_indexed.index:
                    row = df_indexed.loc[file_name]
                    
                    # Crear el diccionario de datos con los campos exactos del CSV
                    data = {
                        "nombre": str(row["Nombre"]) if not pd.isna(row["Nombre"]) else "",
                        "segundo_nombre": str(row["Segundo Nombre"]) if not pd.isna(row["Segundo Nombre"]) else "",
                        "apellido_paterno": str(row["Apellido Paterno"]) if not pd.isna(row["Apellido Paterno"]) else "",
                        "apellido_materno": str(row["Apellido Materno"]) if not pd.isna(row["Apellido Materno"]) else "",
                        "estado": str(row["Estado"]) if not pd.isna(row["Estado"]) else "",
                        "municipio": str(row["Municipio"]) if not pd.isna(row["Municipio"]) else "",
                        "colonia": str(row["Colonia"]) if not pd.isna(row["Colonia"]) else "",
                        "calle": str(row["Calle"]) if not pd.isna(row["Calle"]) else "",
                        "numero_ext": str(row["Numero Ext."]) if not pd.isna(row["Numero Ext."]) else "",
                        "numero_int": str(row["Numero Int."]) if not pd.isna(row["Numero Int."]) else "",
                        "codigo_postal": str(row["Codigo Postal"]) if not pd.isna(row["Codigo Postal"]) else "",
                        "curp": str(row["CURP"]) if not pd.isna(row["CURP"]) else "",
                        "clave_elector": str(row["Clave Elector"]) if not pd.isna(row["Clave Elector"]) else "",
                        "fecha_nacimiento": str(row["Fecha de Nacimiento"]) if not pd.isna(row["Fecha de Nacimiento"]) else "",
                        "seccion": str(row["Seccion"]) if not pd.isna(row["Seccion"]) else "",
                        "vigencia_inicio": str(row["Vigencia Ano Inicio"]) if not pd.isna(row["Vigencia Ano Inicio"]) else "",
                        "vigencia_fin": str(row["Vigencia Ano Final"]) if not pd.isna(row["Vigencia Ano Final"]) else "",
                        "ano_registro": str(row["Ano Registro"]) if not pd.isna(row["Ano Registro"]) else "",
                        "mes_registro": str(row["Mes Registro"]) if not pd.isna(row["Mes Registro"]) else ""
                    }
                    
                    gt_parse = {"gt_parse": data}
                    line = {
                        "file_name": file_name,
                        "ground_truth": json.dumps(gt_parse)
                    }
    
                    with open(metadata_path, "a") as f:
                        f.write(json.dumps(line) + "\n")
                else:
                    print(f"Advertencia: {file_name} no se encuentra en el CSV original")
                    
            except Exception as e:
                print(f"Error al procesar {file_name}: {e}")
    
    print("Regeneración de metadatos completada correctamente.")

# Ejemplo de uso
if __name__ == "__main__":
    csv_path = "datos_ines.csv"  # CSV original con los datos correctos
    base_folder = r"C:\Users\Adrian\Desktop\INEs Dataset"  # Carpeta base
    
    regenerate_metadata_from_csv(csv_path, base_folder)